# Advanced AI Agents with Google Calendar Integration - Lecture 2

Welcome to Lecture 2! 🎉 In this hands-on session, you'll learn how to:

1. 🤖 Build intelligent AI agents using the `openai-agents` library
2. 🔧 Create function tools that agents can use
3. 📅 Integrate AI agents with Google Calendar API
4. 🚀 Build a complete agent-powered scheduling system
5. 💬 Handle natural language processing for calendar events

## Learning Objectives

By the end of this lecture, you will be able to:
- Create sophisticated AI agents with custom tools
- Build function decorators for agent capabilities
- Integrate external APIs (Google Calendar) with AI agents
- Handle natural language input for structured data extraction
- Deploy a complete agent-powered application

## Prerequisites
- Completed Lecture 1 (Basic LLM and Calendar API setup)
- Azure OpenAI credentials configured
- Google Calendar API credentials set up
- Basic understanding of async/await in Python

Let's dive into the world of intelligent agents! 🚀

## Section 1: Setting Up Advanced Agents 🔧

In this section, we'll set up the `openai-agents` library and configure it to work with your Azure OpenAI credentials. This is more advanced than the basic LLM calls we used in Lecture 1.

### What are AI Agents?
- **Agents** are AI systems that can use tools to accomplish tasks
- **Function Tools** allow agents to call Python functions
- **Runners** execute agent workflows and handle conversations
- **Sessions** maintain memory across interactions

In [1]:
# 📦 Exercise 1.1: Import and Setup
# Install required packages first (run once)
# !pip install openai-agents google-api-python-client google-auth-httplib2 google-auth-oauthlib

print("📦 Setting up advanced AI agents...")

# Core imports
import asyncio
import os
import json
import datetime
import pickle
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Import the powerful agents library
from agents import (
    Agent,
    Runner,
    function_tool,
    set_default_openai_client,
)
from openai import AsyncAzureOpenAI

# Google Calendar libraries (from Lecture 1)
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

print("✅ All libraries imported successfully!")
print("🎯 Ready to build intelligent agents!")

📦 Setting up advanced AI agents...
✅ All libraries imported successfully!
🎯 Ready to build intelligent agents!


In [2]:
# 🔧 Exercise 1.2: Configure Azure OpenAI for Agents
print("🔧 Configuring Azure OpenAI for agents...")

# Configure Azure OpenAI client (more advanced than Lecture 1)
azure_client = AsyncAzureOpenAI(
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("OPENAI_VERSION_NAME"), 
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
)

# Set the default client for the agents library with tracing disabled
set_default_openai_client(azure_client, use_for_tracing=False)

# Completely disable tracing to avoid API key conflicts
if 'OPENAI_API_KEY' in os.environ:
    del os.environ['OPENAI_API_KEY']

print("✅ Azure OpenAI client configured for agents!")
print(f"🔗 Endpoint: {os.environ.get('AZURE_OPENAI_ENDPOINT')}")
print(f"📝 Model: {os.environ.get('OPENAI_DEPLOYMENT_NAME')}")
print("🚫 Tracing disabled to avoid conflicts")

🔧 Configuring Azure OpenAI for agents...
✅ Azure OpenAI client configured for agents!
🔗 Endpoint: https://pdpjulie-openai.openai.azure.com/
📝 Model: gpt-4o
🚫 Tracing disabled to avoid conflicts


## Section 2: Understanding Function Tools 🛠️

**Function Tools** are the secret sauce of AI agents! They allow your agent to:
- Call Python functions
- Access external APIs
- Perform calculations
- Interact with databases
- And much more!

### The `@function_tool` Decorator
This decorator converts any Python function into a tool that agents can use. The agent can:
1. **See** the function signature and docstring
2. **Understand** what the function does
3. **Call** the function with appropriate parameters
4. **Use** the function's return value in its response

In [3]:
# 🧪 Exercise 2.1: Create Your First Function Tool
print("🛠️ Creating simple function tools...")

@function_tool
async def get_current_date() -> str:
    """Get the current date in YYYY-MM-DD format"""
    return datetime.datetime.now().strftime('%Y-%m-%d')

@function_tool
async def calculate_duration(start_time: str, end_time: str) -> str:
    """
    Calculate duration between two times in HH:MM format
    
    Args:
        start_time: Start time in HH:MM format (24-hour)
        end_time: End time in HH:MM format (24-hour)
        
    Returns:
        Duration as a string (e.g., "2 hours 30 minutes")
    """
    try:
        start_hour, start_min = map(int, start_time.split(':'))
        end_hour, end_min = map(int, end_time.split(':'))
        
        start_total = start_hour * 60 + start_min
        end_total = end_hour * 60 + end_min
        
        if end_total < start_total:  # Next day
            end_total += 24 * 60
            
        duration_min = end_total - start_total
        hours = duration_min // 60
        minutes = duration_min % 60
        
        if hours > 0 and minutes > 0:
            return f"{hours} hours {minutes} minutes"
        elif hours > 0:
            return f"{hours} hours"
        else:
            return f"{minutes} minutes"
            
    except Exception as e:
        return f"Error calculating duration: {str(e)}"

@function_tool
async def get_weather_mock(city: str) -> str:
    """Get mock weather information for a city"""
    weather_data = {
        "brussels": "🌤️ Partly cloudy, 18°C",
        "london": "🌧️ Rainy, 15°C", 
        "paris": "☀️ Sunny, 22°C",
        "new york": "🌥️ Cloudy, 20°C"
    }
    return weather_data.get(city.lower(), f"🌍 Weather data not available for {city}")

print("✅ Function tools created!")
print("🔧 Tools available:")
print("  - get_current_date(): Returns today's date")
print("  - calculate_duration(): Calculates time between two times")  
print("  - get_weather_mock(): Returns mock weather data")

🛠️ Creating simple function tools...
✅ Function tools created!
🔧 Tools available:
  - get_current_date(): Returns today's date
  - calculate_duration(): Calculates time between two times
  - get_weather_mock(): Returns mock weather data


In [ ]:
# 🧪 Exercise 2.2: Test Function Tools with a Simple Agent
print("🤖 Creating a simple agent to test our tools...")

# Create a simple agent with our function tools
simple_agent = Agent(
    name="Helper Assistant",
    model=os.environ.get("OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    instructions="""You are a helpful assistant with access to useful tools.
    
    Use the available tools to help users with:
    - Getting current date information
    - Calculating time durations
    - Getting weather information
    
    Always use the tools when relevant and explain what you're doing.""",
    tools=[get_current_date, calculate_duration, get_weather_mock]
)

# Test the agent
async def test_simple_agent():
    print("🧪 Testing simple agent with tools...")
    
    # Test 1: Date query
    print("\n1️⃣ Test: What's today's date?")
    result1 = await Runner().run(
        starting_agent=simple_agent,
        input="What's today's date?",
        max_turns=2
    )
    print(f"📅 Agent: {result1.final_output}")
    
    # Test 2: Duration calculation
    print("\n2️⃣ Test: How long is a meeting from 14:30 to 16:45?")
    result2 = await Runner().run(
        starting_agent=simple_agent,
        input="How long is a meeting from 14:30 to 16:45?",
        max_turns=2
    )
    print(f"⏰ Agent: {result2.final_output}")
    
    # Test 3: Weather query
    print("\n3️⃣ Test: What's the weather in Brussels?")
    result3 = await Runner().run(
        starting_agent=simple_agent,
        input="What's the weather like in Brussels?",
        max_turns=2
    )
    print(f"🌤️ Agent: {result3.final_output}")

# Run the tests
print("🚀 Running agent tests...")
await test_simple_agent()
print("\n✅ All tests completed! The agent successfully used our function tools!")

🤖 Creating a simple agent to test our tools...
🚀 Running agent tests...
🧪 Testing simple agent with tools...

1️⃣ Test: What's today's date?
📅 Agent: Today's date is October 28, 2025.

2️⃣ Test: How long is a meeting from 14:30 to 16:45?


OPENAI_API_KEY is not set, skipping trace export


⏰ Agent: The meeting is 2 hours and 15 minutes long.

3️⃣ Test: What's the weather in Brussels?
🌤️ Agent: The weather in Brussels is currently partly cloudy with a temperature of 18°C.

✅ All tests completed! The agent successfully used our function tools!


OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


## Section 3: Google Calendar Integration 📅

Now let's integrate Google Calendar with our agents! We'll reuse the calendar functions from Lecture 1 but turn them into powerful agent tools.

### Key Concepts:
- **Service Authentication**: Reusing your existing Google Calendar credentials
- **Function Tool Conversion**: Turning calendar functions into agent tools
- **Error Handling**: Making tools robust for agent use
- **Return Values**: Providing clear feedback to the agent

In [9]:
# 🔧 Exercise 3.1: Set up Google Calendar Service
print("📅 Setting up Google Calendar integration...")

# Google Calendar setup (from Lecture 1)
SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_calendar_service():
    """Get authenticated Google Calendar service (reused from Lecture 1)"""
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Get credentials from environment
            Client_id = os.getenv("GOOGLE_CLIENT_ID")
            client_secret = os.getenv("GOOGLE_CLIENT_SECRET")
            
            flow = InstalledAppFlow.from_client_config(
                {
                    "installed": {
                        "client_id": Client_id,
                        "client_secret": client_secret,
                        "auth_uri": "https://accounts.google.com/o/oauth2/auth",
                        "token_uri": "https://oauth2.googleapis.com/token"
                    }
                },
                SCOPES
            )
            
            # This will open a browser for authentication
            print("🔐 Opening browser for Google Calendar authentication...")
            creds = flow.run_local_server(port=0)

        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)
    return service

# Test the calendar service
try:
    calendar_service = get_calendar_service()
    print("✅ Google Calendar service authenticated successfully!")
    print("🔗 Ready to integrate with AI agents!")
except Exception as e:
    print(f"❌ Calendar authentication failed: {e}")
    print("💡 Make sure your Google credentials are set up correctly")

📅 Setting up Google Calendar integration...
✅ Google Calendar service authenticated successfully!
🔗 Ready to integrate with AI agents!


In [10]:
# 🛠️ Exercise 3.2: Create Calendar Function Tools
print("🛠️ Creating powerful calendar function tools...")

@function_tool
async def create_calendar_event(
    title: str,
    date: str,
    start_time: str,
    end_time: str,
    location: str = "",
    description: str = ""
) -> str:
    """
    Create a Google Calendar event
    
    Args:
        title: Event title/name (required)
        date: Event date in YYYY-MM-DD format (required)
        start_time: Start time in HH:MM format, 24-hour (required)
        end_time: End time in HH:MM format, 24-hour (required)
        location: Event location (optional)
        description: Event description (optional)
        
    Returns:
        Success message with event details and link
    """
    try:
        # Get calendar service
        service = get_calendar_service()
        
        # Create datetime strings (ISO format for Google Calendar)
        start_datetime = f"{date}T{start_time}:00"
        end_datetime = f"{date}T{end_time}:00"
        
        # Build the event structure
        event = {
            'summary': title,
            'location': location,
            'description': description or f"Event: {title}",
            'start': {
                'dateTime': start_datetime,
                'timeZone': 'Europe/Brussels',
            },
            'end': {
                'dateTime': end_datetime,
                'timeZone': 'Europe/Brussels',
            },
            'reminders': {
                'useDefault': False,
                'overrides': [
                    {'method': 'popup', 'minutes': 15},
                    {'method': 'email', 'minutes': 60},
                ],
            },
        }
        
        # Create the event
        created_event = service.events().insert(calendarId='primary', body=event).execute()
        event_link = created_event.get('htmlLink', 'No link available')
        
        return f"✅ Successfully created event: '{title}' on {date} from {start_time} to {end_time}. Event link: {event_link}"
        
    except Exception as e:
        return f"❌ Failed to create event: {str(e)}"

@function_tool
async def list_upcoming_events(max_results: int = 5) -> str:
    """
    List upcoming events from Google Calendar
    
    Args:
        max_results: Maximum number of events to return (default: 5)
        
    Returns:
        List of upcoming events with details
    """
    try:
        service = get_calendar_service()
        
        # Get current time in RFC3339 format
        now = datetime.datetime.utcnow().isoformat() + 'Z'
        
        events_result = service.events().list(
            calendarId='primary',
            timeMin=now,
            maxResults=max_results,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        
        if not events:
            return "📅 No upcoming events found."
        
        result = f"📅 Next {len(events)} upcoming events:\n"
        for i, event in enumerate(events, 1):
            start = event['start'].get('dateTime', event['start'].get('date'))
            title = event.get('summary', 'No title')
            location = event.get('location', 'No location')
            
            # Format the datetime for display
            if 'T' in start:
                event_datetime = datetime.datetime.fromisoformat(start.replace('Z', '+00:00'))
                formatted_time = event_datetime.strftime('%Y-%m-%d at %H:%M')
            else:
                formatted_time = start
                
            result += f"{i}. {title} - {formatted_time}"
            if location:
                result += f" at {location}"
            result += "\n"
            
        return result
        
    except Exception as e:
        return f"❌ Failed to list events: {str(e)}"

print("✅ Calendar function tools created!")
print("🔧 Tools available:")
print("  - create_calendar_event(): Creates events in Google Calendar")
print("  - list_upcoming_events(): Lists upcoming calendar events")

🛠️ Creating powerful calendar function tools...
✅ Calendar function tools created!
🔧 Tools available:
  - create_calendar_event(): Creates events in Google Calendar
  - list_upcoming_events(): Lists upcoming calendar events


## Section 4: Building the Calendar Scheduling Agent 🤖

Now comes the exciting part! We'll create an intelligent agent that can:
- 🧠 Understand natural language requests
- 📊 Extract structured data from free text
- 📅 Schedule events in Google Calendar
- 💬 Provide helpful feedback and confirmations

### Agent Design Principles:
1. **Clear Instructions**: Tell the agent exactly what it should do
2. **Tool Integration**: Give the agent access to the right tools
3. **Error Handling**: Guide the agent on how to handle problems
4. **User Experience**: Make interactions smooth and intuitive

In [11]:
# 🤖 Exercise 4.1: Create the Master Calendar Agent
print("🤖 Creating the intelligent Calendar Scheduling Agent...")

calendar_agent = Agent(
    name="Calendar Scheduler Pro",
    model=os.environ.get("OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    instructions="""You are an expert calendar scheduling assistant with access to Google Calendar.

Your primary job is to help users schedule events efficiently and accurately.

WHEN SCHEDULING EVENTS:
1. Extract key details: title, date, start time, end time, location, description
2. If ANY required information is missing (title, date, start_time, end_time), ask the user for clarification
3. Use ONLY the create_calendar_event function to schedule events
4. Always confirm successful scheduling with event details

FORMATTING REQUIREMENTS:
- Dates: Use YYYY-MM-DD format (e.g., 2025-10-29)
- Times: Use 24-hour format HH:MM (e.g., 14:30 for 2:30 PM)
- If user gives 12-hour format, convert to 24-hour

HELPFUL FEATURES:
- Use get_current_date() to handle relative dates like "tomorrow", "next Monday"
- Use list_upcoming_events() to check for scheduling conflicts
- Use calculate_duration() to help with time planning

COMMUNICATION STYLE:
- Be friendly and professional
- Always confirm details before creating events
- Provide clear success messages with event links
- Ask clarifying questions when information is incomplete

Remember: You have the power to actually create real calendar events!""",
    tools=[
        create_calendar_event, 
        list_upcoming_events, 
        get_current_date, 
        calculate_duration
    ]
)

print("✅ Calendar Scheduling Agent created!")
print("🧠 Agent capabilities:")
print("  - Natural language understanding")
print("  - Event extraction and validation")
print("  - Google Calendar integration")
print("  - Smart date/time handling")
print("  - Conflict checking")
print("🚀 Ready to schedule events!")

🤖 Creating the intelligent Calendar Scheduling Agent...
✅ Calendar Scheduling Agent created!
🧠 Agent capabilities:
  - Natural language understanding
  - Event extraction and validation
  - Google Calendar integration
  - Smart date/time handling
  - Conflict checking
🚀 Ready to schedule events!


In [12]:
# 🧪 Exercise 4.2: Test the Calendar Agent
print("🧪 Testing the Calendar Scheduling Agent...")

async def test_calendar_agent():
    """Test the calendar agent with various scheduling scenarios"""
    
    print("="*60)
    print("CALENDAR AGENT TESTING SUITE")
    print("="*60)
    
    # Test scenarios - modify these as needed!
    test_scenarios = [
        {
            "name": "Simple Event",
            "input": "Schedule a team meeting tomorrow from 2:00 PM to 3:30 PM in Conference Room A"
        },
        {
            "name": "Detailed Event", 
            "input": "Create an event for 'Machine Learning Workshop' on 2025-10-30 from 09:00 to 12:00 at University Campus, Room 101. This is a hands-on workshop about neural networks."
        },
        {
            "name": "Check Upcoming Events",
            "input": "What are my upcoming events this week?"
        }
    ]
    
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n🔍 TEST {i}: {scenario['name']}")
        print(f"📝 Request: {scenario['input']}")
        print("-" * 50)
        
        try:
            result = await Runner().run(
                starting_agent=calendar_agent,
                input=scenario['input'],
                max_turns=5  # Allow multiple turns for clarification
            )
            
            print(f"🤖 Agent Response:")
            print(f"{result.final_output}")
            
        except Exception as e:
            print(f"❌ Error in test {i}: {e}")
        
        print("\n" + "="*50)

# Run the comprehensive test
print("🚀 Starting comprehensive calendar agent tests...")
await test_calendar_agent()
print("\n🎉 All tests completed!")

🧪 Testing the Calendar Scheduling Agent...
🚀 Starting comprehensive calendar agent tests...
CALENDAR AGENT TESTING SUITE

🔍 TEST 1: Simple Event
📝 Request: Schedule a team meeting tomorrow from 2:00 PM to 3:30 PM in Conference Room A
--------------------------------------------------
🤖 Agent Response:
Your team meeting has been successfully scheduled! Here are the details:

- **Title:** Team Meeting  
- **Date:** 2025-10-29 (Tomorrow)  
- **Time:** 14:00 to 15:30 (2:00 PM - 3:30 PM)  
- **Location:** Conference Room A  

You can access the event using [this link](https://www.google.com/calendar/event?eid=bXQ5bmE3NHFjb3M2cGw0NmJyMWplMW9uYmsganVsaWV2YW5hY2tlcmUxNEBt).

Let me know if you need further assistance! 😊


🔍 TEST 2: Detailed Event
📝 Request: Create an event for 'Machine Learning Workshop' on 2025-10-30 from 09:00 to 12:00 at University Campus, Room 101. This is a hands-on workshop about neural networks.
--------------------------------------------------
🤖 Agent Response:
Your e

C:\Users\JulieVanackere\AppData\Local\Temp\ipykernel_35152\1962512893.py:81: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'


🤖 Agent Response:
Here are your upcoming events this week:

1. **Team Meeting**  
   - **Date:** 2025-10-29  
   - **Time:** 14:00  
   - **Location:** Conference Room A  

2. **Machine Learning Workshop**  
   - **Date:** 2025-10-30  
   - **Time:** 09:00  
   - **Location:** University Campus, Room 101  

The following events are later but still worth noting:

3. **Bé jarig** (2025-11-01)  
4. **Sisterdag** (2025-11-01)  
5. **Swimming ('Zwemmen')** (2025-11-04 at 20:30)

Let me know if you’d like details or changes for any event!


🎉 All tests completed!


## Section 5: Interactive Calendar Assistant 💬

Now let's create an interactive version where you can chat with the agent and schedule your own events!

### Interactive Features:
- **Conversational Interface**: Chat naturally with the agent
- **Real-time Scheduling**: Create actual calendar events
- **Smart Validation**: Agent asks for missing information  
- **Conflict Detection**: Check for scheduling conflicts
- **Flexible Input**: Handle various date/time formats

In [ ]:
# 💬 Exercise 5.1: Interactive Calendar Assistant
print("💬 Setting up Interactive Calendar Assistant...")

async def interactive_calendar_chat():
    """Interactive chat with the calendar agent"""
    
    print("🤖 INTERACTIVE CALENDAR ASSISTANT")
    print("="*50)
    print("Hi! I'm your AI Calendar Assistant. I can help you:")
    print("📅 Schedule new events")
    print("📋 Check upcoming events") 
    print("⏰ Calculate meeting durations")
    print("📍 Add locations and descriptions")
    print("\nJust tell me what you'd like to do in natural language!")
    print("Type 'quit' to exit.")
    print("="*50)
    
    # Example requests you can try:
    example_requests = [
        "Schedule a dentist appointment next Tuesday at 2 PM",
        "What meetings do I have this week?",
        "Book a 1-hour coffee meeting with Sarah tomorrow at 10 AM",
        "Create a workout session for Friday evening",
        "Add a team standup every Monday at 9 AM"
    ]
    
    print("💡 Example requests you can try:")
    for i, example in enumerate(example_requests, 1):
        print(f"   {i}. {example}")
    print()
    
    # For notebook testing, let's simulate a conversation
    # In a real interactive scenario, you'd get user input
    
    test_conversations = [
        "What are my upcoming events?",
        "Schedule a Python study session tomorrow from 7 PM to 9 PM at home"
    ]
    
    for i, user_input in enumerate(test_conversations, 1):
        print(f"👤 User: {user_input}")
        
        try:
            result = await Runner().run(
                starting_agent=calendar_agent,
                input=user_input,
                max_turns=3
            )
            
            print(f"🤖 Assistant: {result.final_output}")
            print("-" * 40)
            
        except Exception as e:
            print(f"❌ Error: {e}")
        
        print()

# Run the interactive session
print("🚀 Starting interactive calendar assistant...")
await interactive_calendar_chat()

print("✅ Interactive session complete!")
print("💡 Tip: Modify the 'test_conversations' list above to try your own requests!")

## Section 6: Advanced Features & Extensions 🚀

Let's explore some advanced features you can add to your calendar agent!

### Advanced Features We'll Implement:
1. **🔍 Event Search**: Find specific events in your calendar
2. **📊 Schedule Analysis**: Get insights about your calendar
3. **🔄 Event Modification**: Update or cancel existing events
4. **📋 Bulk Operations**: Handle multiple events at once
5. **🧠 Smart Suggestions**: Get scheduling recommendations

In [14]:
# 🚀 Exercise 6.1: Advanced Calendar Function Tools
print("🚀 Creating advanced calendar function tools...")

@function_tool
async def analyze_schedule(time_period: str = "week") -> str:
    """
    Analyze calendar schedule and provide insights
    
    Args:
        time_period: Time period to analyze ('week', 'month', or 'day')
        
    Returns:
        Schedule analysis with statistics and insights
    """
    try:
        service = get_calendar_service()
        
        # Calculate time range based on period
        now = datetime.datetime.utcnow()
        if time_period.lower() == "day":
            time_min = now.replace(hour=0, minute=0, second=0, microsecond=0)
            time_max = time_min + datetime.timedelta(days=1)
            period_name = "today"
        elif time_period.lower() == "week": 
            days_since_monday = now.weekday()
            time_min = (now - datetime.timedelta(days=days_since_monday)).replace(hour=0, minute=0, second=0, microsecond=0)
            time_max = time_min + datetime.timedelta(days=7)
            period_name = "this week"
        else:  # month
            time_min = now.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
            next_month = time_min.replace(month=time_min.month % 12 + 1) if time_min.month < 12 else time_min.replace(year=time_min.year + 1, month=1)
            time_max = next_month
            period_name = "this month"
        
        # Get events in the time range
        events_result = service.events().list(
            calendarId='primary',
            timeMin=time_min.isoformat() + 'Z',
            timeMax=time_max.isoformat() + 'Z',
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        
        if not events:
            return f"📊 No events found for {period_name}. Your schedule is completely free!"
        
        # Analyze the events
        total_events = len(events)
        total_duration = 0
        event_types = {}
        daily_counts = {}
        
        for event in events:
            # Count events by day
            start_str = event['start'].get('dateTime', event['start'].get('date'))
            if 'T' in start_str:
                event_date = datetime.datetime.fromisoformat(start_str.replace('Z', '+00:00')).date()
                daily_counts[event_date] = daily_counts.get(event_date, 0) + 1
                
                # Calculate duration for timed events
                end_str = event['end'].get('dateTime', event['end'].get('date'))
                if 'T' in end_str:
                    start_time = datetime.datetime.fromisoformat(start_str.replace('Z', '+00:00'))
                    end_time = datetime.datetime.fromisoformat(end_str.replace('Z', '+00:00'))
                    duration = (end_time - start_time).total_seconds() / 3600  # hours
                    total_duration += duration
            
            # Categorize events (simple keyword-based)
            title = event.get('summary', 'Unknown').lower()
            if any(word in title for word in ['meeting', 'call', 'standup']):
                event_types['Meetings'] = event_types.get('Meetings', 0) + 1
            elif any(word in title for word in ['workshop', 'training', 'lecture', 'class']):
                event_types['Learning'] = event_types.get('Learning', 0) + 1
            elif any(word in title for word in ['appointment', 'doctor', 'dentist']):
                event_types['Appointments'] = event_types.get('Appointments', 0) + 1
            else:
                event_types['Other'] = event_types.get('Other', 0) + 1
        
        # Create analysis report
        busiest_day = max(daily_counts.items(), key=lambda x: x[1]) if daily_counts else None
        
        analysis = f"📊 SCHEDULE ANALYSIS FOR {period_name.upper()}\\n"
        analysis += f"{'='*40}\\n"
        analysis += f"📅 Total Events: {total_events}\\n"
        analysis += f"⏰ Total Scheduled Time: {total_duration:.1f} hours\\n"
        
        if busiest_day:
            analysis += f"🔥 Busiest Day: {busiest_day[0]} ({busiest_day[1]} events)\\n"
        
        analysis += f"\\n📋 Event Categories:\\n"
        for category, count in event_types.items():
            percentage = (count / total_events) * 100
            analysis += f"  • {category}: {count} events ({percentage:.1f}%)\\n"
        
        return analysis
        
    except Exception as e:
        return f"❌ Failed to analyze schedule: {str(e)}"

@function_tool 
async def find_free_time(date: str, duration_hours: int = 1) -> str:
    """
    Find available time slots on a specific date
    
    Args:
        date: Date to check in YYYY-MM-DD format
        duration_hours: Required duration in hours (default: 1)
        
    Returns:
        List of available time slots
    """
    try:
        service = get_calendar_service()
        
        # Get events for the specified date
        start_of_day = f"{date}T00:00:00Z"
        end_of_day = f"{date}T23:59:59Z"
        
        events_result = service.events().list(
            calendarId='primary',
            timeMin=start_of_day,
            timeMax=end_of_day,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        
        # Define working hours (9 AM to 6 PM)
        work_start = 9
        work_end = 18
        
        # Create list of busy times
        busy_times = []
        for event in events:
            start_str = event['start'].get('dateTime')
            end_str = event['end'].get('dateTime') 
            
            if start_str and end_str:
                start_time = datetime.datetime.fromisoformat(start_str.replace('Z', '+00:00'))
                end_time = datetime.datetime.fromisoformat(end_str.replace('Z', '+00:00'))
                busy_times.append((start_time.hour + start_time.minute/60, end_time.hour + end_time.minute/60))
        
        # Sort busy times
        busy_times.sort()
        
        # Find free slots
        free_slots = []
        current_time = work_start
        
        for start_busy, end_busy in busy_times:
            if start_busy > current_time + duration_hours:
                free_slots.append((current_time, start_busy))
            current_time = max(current_time, end_busy)
        
        # Check for time after last meeting
        if current_time + duration_hours <= work_end:
            free_slots.append((current_time, work_end))
        
        if not free_slots:
            return f"❌ No {duration_hours}-hour slots available on {date} during working hours (9 AM - 6 PM)"
        
        result = f"🕒 Available {duration_hours}-hour time slots on {date}:\\n"
        for start, end in free_slots:
            start_formatted = f"{int(start):02d}:{int((start % 1) * 60):02d}"
            end_formatted = f"{int(min(end, start + duration_hours)):02d}:{int((min(end, start + duration_hours) % 1) * 60):02d}"
            result += f"  • {start_formatted} - {end_formatted}\\n"
        
        return result
        
    except Exception as e:
        return f"❌ Failed to find free time: {str(e)}"

print("✅ Advanced calendar tools created!")
print("🔧 New tools available:")
print("  - analyze_schedule(): Provides schedule insights and statistics")
print("  - find_free_time(): Finds available time slots for scheduling")

🚀 Creating advanced calendar function tools...
✅ Advanced calendar tools created!
🔧 New tools available:
  - analyze_schedule(): Provides schedule insights and statistics
  - find_free_time(): Finds available time slots for scheduling


In [17]:
# 🚀 Exercise 6.2: Creating an Advanced Agent with New Tools
print("🤖 Creating advanced calendar agent...")

# Create an enhanced agent with all our tools
advanced_calendar_agent = Agent(
    name="Calendar Analytics Guru",
    model=os.environ.get("OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    instructions="""
    You are an advanced AI Calendar Assistant with powerful analytics and scheduling capabilities. 
    
    Your enhanced capabilities include:
    - Creating and listing calendar events
    - Analyzing schedule patterns and providing insights
    - Finding optimal time slots for meetings
    - Providing detailed schedule statistics
    
    Key behaviors:
    - Always be helpful and proactive in suggesting improvements
    - Provide detailed analysis when asked about schedules
    - Use natural language to explain complex scheduling data
    - Offer alternatives when requested times are unavailable
    - Be specific about time formats and confirmations
    
    Always format your responses clearly and include relevant emojis for better user experience.
    """,
    tools=[
        get_current_date,
        create_calendar_event, 
        list_upcoming_events,
        analyze_schedule,
        find_free_time
    ]
)

print("✅ Advanced calendar agent created with enhanced capabilities!")
print("🎯 New features:")
print("  - Schedule analysis and insights")
print("  - Free time slot detection")
print("  - Smart scheduling recommendations")

🤖 Creating advanced calendar agent...
✅ Advanced calendar agent created with enhanced capabilities!
🎯 New features:
  - Schedule analysis and insights
  - Free time slot detection
  - Smart scheduling recommendations


In [21]:
# 🧪 Exercise 6.3: Testing Advanced Features
print("🧪 Testing advanced calendar features...")

# Test 1: Schedule Analysis
print("\\n📊 Testing schedule analysis...")
try:
    response = await Runner().run(
                starting_agent=advanced_calendar_agent,
                input="Can you analyze my schedule for this week and give me insights?",
                max_turns=3
            )
            
    print("Response:", response.final_output)
except Exception as e:
    print(f"❌ Error: {e}")

print("\\n" + "="*50)

# Test 2: Finding Free Time
print("\\n🕒 Testing free time finder...")
try:
    # Get tomorrow's date
    tomorrow = (datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    response = await Runner().run(
                starting_agent=advanced_calendar_agent,
                input=f"Can you find me a 2-hour free slot on {tomorrow}?",
                max_turns=3
            )
    print("Response:", response.final_output)
except Exception as e:
    print(f"❌ Error: {e}")

print("\\n" + "="*50)

# Test 3: Smart Scheduling with Analysis
print("\\n🎯 Testing smart scheduling...")
try:
    response = await Runner().run(
        starting_agent=advanced_calendar_agent,
        input="I need to schedule a 1-hour team meeting sometime next week. "
              "Can you analyze my current schedule and suggest the best time?",
        max_turns=5
    )
    print("Response:", response.final_output)
except Exception as e:
    print(f"❌ Error: {e}")

print("\\n✅ Advanced features testing complete!")

🧪 Testing advanced calendar features...
\n📊 Testing schedule analysis...


C:\Users\JulieVanackere\AppData\Local\Temp\ipykernel_35152\94868582.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


Response: Here’s your **weekly schedule analysis** 📅:

- **Total Events:** 2
- **Total Scheduled Time:** 0 hours (currently you're not dedicating much time to events)
- **Event Categories:**  
   - **Other:** 2 events (100%)  

🧠 **Insights:**  
- Your schedule seems light this week, giving you significant free time for planning additional tasks or meetings.  
- If you aim to improve productivity, consider scheduling focused blocks for work, hobbies, or personal activities.  
- Let me know if you want me to help find opportunities to balance your time better! ⚖️
\n==================================================
\n🕒 Testing free time finder...
Response: You have a free 2-hour time slot available on **October 29, 2025**, from **9:00 AM to 11:00 AM**. ⏳ Let me know if you’d like to schedule anything during this time! 😊
\n==================================================
\n🎯 Testing smart scheduling...
Response: You have a free 2-hour time slot available on **October 29, 2025**, from *

## Section 7: Interactive Chat Interface 💬

Now let's create an interactive chat interface where you can have natural conversations with your calendar agent! This section demonstrates how to build a simple but effective chat loop.

In [26]:
# 💬 Exercise 7.1: Interactive Chat Interface
print("💬 Creating interactive chat interface...")

async def chat_with_agent():
    """Interactive chat interface with the calendar agent"""
    print("🤖 Welcome to your AI Calendar Assistant!")
    print("📝 You can ask me to:")
    print("   • Create calendar events")
    print("   • List upcoming events") 
    print("   • Analyze your schedule")
    print("   • Find free time slots")
    print("   • Get smart scheduling suggestions")
    print("\\n💡 Try asking things like:")
    print("   - 'Create a meeting tomorrow at 2 PM'")
    print("   - 'What do I have scheduled this week?'")
    print("   - 'Analyze my schedule for insights'")
    print("   - 'Find me a free 2-hour slot tomorrow'")
    print("\\n⚠️  Type 'quit', 'exit', or 'bye' to end the conversation\\n")
    
    conversation_history = []
    
    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()
            
            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'bye', 'q']:
                print("\\n🤖 Calendar Agent: Goodbye! Have a great day! 👋")
                break
            
            if not user_input:
                continue
            
            # Add user message to history
            conversation_history.append(f"User: {user_input}")
            
            # Get agent response
            print("\\n🤖 Calendar Agent: ", end="", flush=True)
            
            # Create a runner for streaming responses
            result = await Runner().run(
                starting_agent=advanced_calendar_agent,
                input=user_input,
                max_turns=3
            )
            agent_response = result.final_output
            print(agent_response)
            
            # Add agent response to history
            conversation_history.append(f"Agent: {agent_response}")
            print("\\n" + "-"*50 + "\\n")
            
        except KeyboardInterrupt:
            print("\\n\\n🤖 Calendar Agent: Chat interrupted. Goodbye! 👋")
            break
        except Exception as e:
            print(f"\\n❌ Error: {e}")
            print("Please try again or type 'quit' to exit.\\n")

# Note: The chat interface is ready to use!
# Uncomment the next line to start an interactive chat session:
await chat_with_agent()

print("✅ Interactive chat interface created!")
print("🎯 Uncomment the line above to start chatting with your agent!")
print("💡 Note: This function requires 'await' since it's async")
print("💡 Note: This function requires 'await' since it's async")

💬 Creating interactive chat interface...
🤖 Welcome to your AI Calendar Assistant!
📝 You can ask me to:
   • Create calendar events
   • List upcoming events
   • Analyze your schedule
   • Find free time slots
   • Get smart scheduling suggestions
\n💡 Try asking things like:
   - 'Create a meeting tomorrow at 2 PM'
   - 'What do I have scheduled this week?'
   - 'Analyze my schedule for insights'
   - 'Find me a free 2-hour slot tomorrow'
\n⚠️  Type 'quit', 'exit', or 'bye' to end the conversation\n
\n🤖 Calendar Agent: \n🤖 Calendar Agent: 

C:\Users\JulieVanackere\AppData\Local\Temp\ipykernel_35152\94868582.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
C:\Users\JulieVanackere\AppData\Local\Temp\ipykernel_35152\1962512893.py:81: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'
C:\Users\JulieVanackere\AppData\Local\Temp\ipykernel_35152\1962512893.py:81: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'


🗓️ Here's what you have scheduled for this week:

### This Week's Events:
1. **Bé jarig**  
   📆 Date: November 1, 2025  
   📍 Location: Not specified  

2. **Sisterdag**  
   📆 Date: November 1, 2025  
   📍 Location: Not specified  

### Overview:
You have two events on **November 1, 2025**. Both are marked without specific locations.

If you'd like to schedule anything else or need reminders, let me know! 😊
\n--------------------------------------------------\n
\n🤖 Calendar Agent: Goodbye! Have a great day! 👋
✅ Interactive chat interface created!
🎯 Uncomment the line above to start chatting with your agent!
💡 Note: This function requires 'await' since it's async
💡 Note: This function requires 'await' since it's async
\n🤖 Calendar Agent: Goodbye! Have a great day! 👋
✅ Interactive chat interface created!
🎯 Uncomment the line above to start chatting with your agent!
💡 Note: This function requires 'await' since it's async
💡 Note: This function requires 'await' since it's async


## Section 8: Summary & Next Steps 🎯

Congratulations! You've successfully built a complete AI-powered calendar assistant using the `openai-agents` library. Let's review what we've accomplished and explore future enhancements.

In [ ]:
# 📚 Learning Summary
print("🎯 LECTURE 2 COMPLETION SUMMARY")
print("="*50)

learning_objectives = [
    "✅ Understanding AI Agents and their components",
    "✅ Setting up Azure OpenAI with openai-agents library", 
    "✅ Creating function tools with @function_tool decorator",
    "✅ Integrating Google Calendar API with agents",
    "✅ Building basic calendar management tools",
    "✅ Implementing advanced scheduling analytics",
    "✅ Creating interactive chat interfaces",
    "✅ Testing and debugging agent systems"
]

print("\\n🎓 What You've Learned:")
for objective in learning_objectives:
    print(f"  {objective}")

key_concepts = [
    "Agent Architecture: Models + Instructions + Functions",
    "Function Tools: @function_tool decorator for agent capabilities",
    "API Integration: Connecting external services to agents",
    "Natural Language Processing: Converting human requests to actions",
    "Error Handling: Robust agent behavior with try/catch patterns",
    "Streaming Responses: Real-time agent communication",
    "Interactive Interfaces: Building chat-based applications"
]

print("\\n🧠 Key Concepts Mastered:")
for i, concept in enumerate(key_concepts, 1):
    print(f"  {i}. {concept}")

print("\\n🛠️ Your Complete Agent Toolkit:")
function_tools = [
    "get_current_date() - Date and time utilities",
    "create_calendar_event() - Event creation with smart scheduling", 
    "list_upcoming_events() - Calendar querying and filtering",
    "analyze_schedule() - Schedule insights and analytics",
    "find_free_time() - Intelligent time slot detection"
]

for tool in function_tools:
    print(f"  🔧 {tool}")

print("\\n🎯 Agent Capabilities Achieved:")
capabilities = [
    "Natural language calendar management",
    "Smart scheduling with conflict detection", 
    "Schedule analysis and optimization",
    "Interactive conversational interface",
    "Robust error handling and user feedback"
]

for capability in capabilities:
    print(f"  🚀 {capability}")

print("\\n" + "="*50)
print("🏆 Congratulations! You've built a production-ready AI calendar assistant!")

In [ ]:
# 🚀 Next Steps & Enhancements
print("🚀 SUGGESTED NEXT STEPS & ENHANCEMENTS")
print("="*50)

beginner_enhancements = [
    "Add event modification (update/delete events)",
    "Implement recurring event creation",
    "Add timezone support for global scheduling",
    "Create event reminder notifications",
    "Build calendar sharing capabilities"
]

intermediate_enhancements = [
    "Integrate with multiple calendar services (Outlook, Apple)",
    "Add natural language date parsing (\"next Tuesday\", \"in 2 weeks\")",
    "Implement smart meeting room booking",
    "Create automated scheduling for routine tasks",
    "Build conflict resolution with alternative suggestions"
]

advanced_enhancements = [
    "Multi-agent collaboration (scheduling agent + email agent)",
    "Machine learning for optimal meeting time prediction",
    "Integration with video conferencing APIs (Zoom, Teams)",
    "Advanced analytics with visualization dashboards",
    "Voice-enabled calendar management"
]

print("\\n🟢 Beginner Level Enhancements:")
for i, enhancement in enumerate(beginner_enhancements, 1):
    print(f"  {i}. {enhancement}")

print("\\n🟡 Intermediate Level Enhancements:")
for i, enhancement in enumerate(intermediate_enhancements, 1):
    print(f"  {i}. {enhancement}")

print("\\n🔴 Advanced Level Enhancements:")
for i, enhancement in enumerate(advanced_enhancements, 1):
    print(f"  {i}. {enhancement}")

print("\\n🎯 Recommended Learning Path:")
learning_path = [
    "Practice with more complex scheduling scenarios",
    "Explore other openai-agents features (persistent conversations)",
    "Learn about agent orchestration and multi-agent systems", 
    "Study production deployment patterns for AI agents",
    "Investigate agent monitoring and performance optimization"
]

for i, step in enumerate(learning_path, 1):
    print(f"  {i}. {step}")

print("\\n📚 Additional Resources:")
resources = [
    "OpenAI Agents Documentation: https://github.com/openai/agents",
    "Google Calendar API Guide: https://developers.google.com/calendar",
    "Azure OpenAI Documentation: https://docs.microsoft.com/azure/cognitive-services/openai/",
    "Python Async Programming: https://docs.python.org/3/library/asyncio.html"
]

for resource in resources:
    print(f"  📖 {resource}")

print("\\n" + "="*50)
print("🎓 Keep building and exploring! The world of AI agents is vast and exciting!")

## 🎉 Final Exercise: Build Your Own Agent!

**Challenge**: Now it's your turn! Using what you've learned, create your own AI agent for a different domain. Here are some ideas:

### 💡 Project Ideas:
1. **📧 Email Assistant**: Compose, send, and organize emails
2. **📝 Task Manager**: Create, track, and prioritize tasks
3. **🌤️ Weather Agent**: Provide weather forecasts and alerts
4. **📈 Finance Tracker**: Monitor expenses and create budgets
5. **🎵 Music Manager**: Control music playback and create playlists

### 🛠️ Your Agent Template:
```python
# Your custom agent implementation
@function_tool
async def your_custom_function():
    """Implement your agent's core functionality"""
    pass

your_agent = Agent(
    model="gpt-4o",
    client=client,
    instructions="Your agent's personality and capabilities...",
    functions=[your_custom_function]
)
```

**Good luck and happy coding!** 🚀